In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [3]:
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
bs = BeautifulSoup(page,'xml')

In [4]:
table = bs.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')

In [5]:
data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

In [6]:
df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[(~df['PostalCode'].isnull()) & (df['Borough'] != 'Not assigned')]
df = df.groupby(['PostalCode', 'Borough'])['Neighbourhood'].apply(list)
df = pd.DataFrame(df)
df1 = df.sort_values(by=['Neighbourhood'])
df1 = df1.reset_index()

In [7]:
column_names = ['Neighbourhood'] 
neights1 = pd.DataFrame(columns=column_names)

In [8]:
prueba1 = df1.loc[:,'Neighbourhood']

for row in prueba1:
    prueba11 = ','.join(row)
    neights1 = neights1.append({'Neighbourhood' : prueba11}, ignore_index = True)

In [9]:
torn_data = df1.drop(['Neighbourhood'], axis= 1)
toronto_data = pd.merge(torn_data, neights1, left_index=True, right_index=True)
toronto_data.head()

,PostalCode,Borough,Neighbourhood
0,M5H,Downtown Toronto,"Adelaide,King,Richmond"
1,M1S,Scarborough,Agincourt
2,M1V,Scarborough,"Agincourt North,L'Amoreaux East,Milliken,Steel..."
3,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."
4,M8W,Etobicoke,"Alderwood,Long Branch"


#### In this notebook, I used Beatifullsoup library to scrap the Wikipedia page. Then, I transforme the data in format requiered, cleaning data remove the Borough whit Not assigned and missings

In [10]:
toronto_data.shape

(103, 3)

In [11]:
path ='http://cocl.us/Geospatial_data'
dfl = pd.read_csv(path)
dfl.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
dfc = pd.merge(toronto_data, dfl, left_on ='PostalCode', right_on = 'Postal Code')
dfc.head()

,PostalCode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M5H,Downtown Toronto,"Adelaide,King,Richmond",M5H,43.650571,-79.384568
1,M1S,Scarborough,Agincourt,M1S,43.794200,-79.262029
2,M1V,Scarborough,"Agincourt North,L'Amoreaux East,Milliken,Steel...",M1V,43.815252,-79.284577
3,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",M9V,43.739416,-79.588437
4,M8W,Etobicoke,"Alderwood,Long Branch",M8W,43.602414,-79.543484
